In [ ]:
import pandas as pd
import numpy as np 
from sklearn.datasets import load_files

# Text cleaning and preprocessing
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from tqdm.notebook import tqdm

In [ ]:
df = pd.read_csv("./spam/train.csv")
df.shape

In [ ]:
np.unique(df['Spam/Ham'])

In [ ]:
df.head()

Now we have list of texts, that encoded binary. Using 'decode' is one of possible solutions, but some texts don't allow us to apply decoding. This can be solved by deleting these texts, but because of this, we can lose important information. Instead of this we can do following:

* We should remove all special symbols.
* Remove 'b' in beginning of each text
* Replace all gaps (\t, \n, \r, \f) between words with spaces
* Remove all non-letters characters

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import HashingVectorizer
#tf = TfidfVectorizer()
hvec = HashingVectorizer(n_features=2**9,alternate_sign=False) #***change this logically***

def preProcess(X):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    corpus = []
    for i in tqdm(range(0, len(X))):
        # Remove special symbols
        review = re.sub(r'\\r\\n', ' ', str(X[i]))
        # Remove all symbols except letters
        review = re.sub('[^a-zA-Z]', ' ', review)
        # Replacing all gaps with spaces 
        review = re.sub(r'\s+', ' ', review)                    
        # Remove 'b' in the beginning of each text
        review = re.sub(r'^b\s+', '', review)       

        review = review.lower().split()
        review = [stemmer.stem(word) for word in review if word not in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
    
    # Creating the Bag of Words model
    X = hvec.fit_transform(corpus)
    return X.toarray()

def trainBatch(X, y):
    X = preProcess(X)

    # Splitting data on train and test dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y,  random_state=9, test_size=0.2)
    
    global model
    model = model.partial_fit(X_train, y_train, np.unique(y))
    pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred, pos_label = 'spam')
    recall = recall_score(y_test, pred, pos_label = 'spam')
    conf_m = confusion_matrix(y_test, pred)

    print(f"accuracy: %.3f" %accuracy)
    print(f"precision: %.3f" %precision)
    print(f"recall: %.3f" %recall)
    print(f"confusion matrix: ")
    print(conf_m)

    return accuracy, precision, recall

Trying Naive Bayes model as the likelihood of whether an email is spam or ham is an aposterior probability and they usually show high performance in spam detection.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()

```
accuracy: 0.979
precision: 0.962
recall: 1.000
confusion matrix: 
[[452  21]
 [  0 527]]
```

In [ ]:
def makeBatch(df, batchSize = 500):
    for i in range(len(df)//batchSize):
        X, y = df['Message'][i:i + batchSize + 1], df['Spam/Ham'][i:i + batchSize + 1]
        X.index = range(batchSize + 1)
        y.index = range(batchSize + 1)
        yield X, y

In [ ]:
batch = makeBatch(df, 100)
NBatches = 5

In [ ]:
metrics = ('accuracy', 'precision', 'recall')
history = {metric : [] for metric in metrics}
for batchNum in range(NBatches):
    print(f"------------------------ Batch {batchNum + 1} ------------------------")
    X,y = next(batch)
    scores = trainBatch(X,y)
    for i in range(len(metrics)):
        history[metrics[i]].append(scores[i])
    print("---------------------------------------------------------")

In [ ]:
import matplotlib.pyplot as plt
for metric in metrics:
    plt.plot(history[metric], label = metric)

plt.legend()
plt.grid()
plt.ylabel('Metrics')
plt.xlabel('Number of Batches')
plt.title('Model performance over number of Batches used for training')
plt.show()

In [ ]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(loss='log')

In [ ]:
batch = makeBatch(df)
NBatches = 5

In [ ]:
metrics = ('accuracy', 'precision', 'recall')
history = {metric : [] for metric in metrics}
for batchNum in range(NBatches):
    print(f"------------------------ Batch {batchNum + 1} ------------------------")
    X,y = next(batch)
    scores = trainBatch(X,y)
    for i in range(len(metrics)):
        history[metrics[i]].append(scores[i])
    print("---------------------------------------------------------")

In [ ]:
import matplotlib.pyplot as plt
for metric in metrics:
    plt.plot(history[metric], label = metric)

plt.legend()
plt.grid()
plt.ylabel('Metrics')
plt.xlabel('Number of Batches')
plt.title('Model performance over number of Batches used for training')
plt.show()

In [ ]:
from sklearn.linear_model import Perceptron

model = Perceptron()

In [ ]:
batch = makeBatch(df)
NBatches = 5

In [ ]:
metrics = ('accuracy', 'precision', 'recall')
history = {metric : [] for metric in metrics}
for batchNum in range(NBatches):
    print(f"------------------------ Batch {batchNum + 1} ------------------------")
    X,y = next(batch)
    scores = trainBatch(X,y)
    for i in range(len(metrics)):
        history[metrics[i]].append(scores[i])
    print("---------------------------------------------------------")

In [ ]:
import matplotlib.pyplot as plt
for metric in metrics:
    plt.plot(history[metric], label = metric)

plt.legend()
plt.grid()
plt.ylabel('Metrics')
plt.xlabel('Number of Batches')
plt.title('Model performance over number of Batches used for training')
plt.show()

In [ ]:
batch = makeBatch(df, 10)
NBatches = 5

from sklearn.cluster import MiniBatchKMeans

model = MiniBatchKMeans(n_clusters=2, random_state=0, batch_size=6)

def trainBatch_cluster(X, y):
    X = preProcess(X)
    global model
    model.partial_fit(X)
    pred = model.predict(X)
    y_test = [0 if i == "spam" else 1 for i in y.tolist()]
    
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred, pos_label = 0)
    recall = recall_score(y_test, pred, pos_label = 0)
    conf_m = confusion_matrix(y_test, pred)

    print(f"accuracy: %.3f" %accuracy)
    print(f"precision: %.3f" %precision)
    print(f"recall: %.3f" %recall)
    print(f"confusion matrix: ")
    print(conf_m)

    return accuracy, precision, recall

In [ ]:
metrics = ('accuracy', 'precision', 'recall')
history = {metric : [] for metric in metrics}
for batchNum in range(NBatches):
    print(f"------------------------ Batch {batchNum + 1} ------------------------")
    X,y = next(batch)
    scores = trainBatch_cluster(X, y)
    for i in range(len(metrics)):
        history[metrics[i]].append(scores[i])
    print("---------------------------------------------------------")